In [ ]:
!pip install agentpy
!pip install seaborn
!pip install matplotlib==3.1.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.4 MB/s 
     |████████████████████████████████| 756 kB 7.5 MB/s 
     |████████████████████████████████| 11.2 MB 31.7 MB/s 
     |████████████████████████████████| 957 kB 45.9 MB/s 
     |████████████████████████████████| 76 kB 6.1 MB/s 
     |████████████████████████████████| 52 kB 1.3 MB/s 
     |████████████████████████████████| 115 kB 56.5 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.12.0
    Uninstalling importlib-metadata-4.12.0:
      Successfully uninstalled importlib-metadata-4.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the s

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.1 MB 3.8 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.5.3
    Uninstalling matplotlib-3.5.3:
      Successfully uninstalled matplotlib-3.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
agentpy 0.1.5 requires matplotlib>=3.3.3, but you have matplotlib 3.1.3 which is incompatible.


In [ ]:
import agentpy as ap
import matplotlib.pyplot as plt
import IPython

In [183]:
# Road agent positions

#ROUNDABOUT_X = []
#ROUNDABOUT_Y = []

ROUNDABOUT = []
POSITIONS_X = []
POSITIONS_Y = []


for i in range(0, 27):
    POSITIONS_X.append((13,i))
    POSITIONS_Y.append((i,13))

'''
for i in range(0, 11):
    POSITIONS_X.append((13,i))
for i in range(16, 27):
    POSITIONS_X.append((13,i))

for i in range(17, 27):
    POSITIONS_Y.append((i,13))
for i in range(0,11):
    POSITIONS_Y.append((i,13))

def midPointCircleDraw(x_centre, y_centre, ratio):
        x = ratio
        y = 0
        
        # Printing the initial point the
        # axes after translation
        ROUNDABOUT.append((x + x_centre, y + y_centre))
        print("(", x + x_centre, ", ",
                  y + y_centre, ")",
                  sep = "", end = "")
        
        # When radius is zero only a single
        # point be printed
        '''

'''
        if (ratio > 0) :
        
            print("(", x + x_centre, ", ",
                      -y + y_centre, ")",
                      sep = "", end = "")
            ROUNDABOUT.append((x + x_centre, -y + y_centre))
            print("(", y + x_centre, ", ",
                      x + y_centre, ")",
                      sep = "", end = "")
            ROUNDABOUT.append((y + x_centre, x + y_centre))
            print("(", -y + x_centre, ", ",
                        x + y_centre, ")", sep = "")
            ROUNDABOUT.append((-y + x_centre, x + y_centre))
'''


'''
        # Initializing the value of P
        P = 1 - ratio
        print(P)
 
        while x > y:
        
            y += 1
            
            # Mid-point inside or on the perimeter
            if P <= 0:
                P = P + 2 * y + 1
                
                
            # Mid-point outside the perimeter
            else:        
                x -= 1
                P = P + 2 * y - 2 * x + 1
            
            # All the perimeter points have
            # already been printed
            if (x < y):
                break
            
            # Printing the generated point its reflection
            # in the other octants after translation
        
            ROUNDABOUT.append((x + x_centre, y + y_centre))
            print("(", x + x_centre, ", ",
                     y + y_centre, ")",
                    sep = "", end = "")
            
            ROUNDABOUT.append((-x + x_centre, y + y_centre))
            ROUNDABOUT.append((x + x_centre, -y + y_centre))
            ROUNDABOUT.append((-x + x_centre, -y + y_centre))
            
            # If the generated point on the line x = y then
            # the perimeter points have already been printed
            
            if x != y:
                #print("(", y + x_centre, ", ",
                 #     x + y_centre, ")",
                  #    sep = "", end = "")
                ROUNDABOUT.append((y + x_centre, x + y_centre))

                #print("(", -y + x_centre, ", ",
                #      x + y_centre, ")",
                #      sep = "", end = "")
                ROUNDABOUT.append((-y + x_centre, x + y_centre))

                #print("(", y + x_centre, ", ",
                #      -x + y_centre, ")",
                #      sep = "", end = "")
                ROUNDABOUT.append((y + x_centre, -x + y_centre))

                #print("(", -y + x_centre, ", ",
                #      -x + y_centre, ")",
                #      sep = "", end = "")
                ROUNDABOUT.append((-y + x_centre, -x + y_centre))
'''             
                             
# Driver Code
#if __name__ == '__main__':
        
  # To draw a circle of radius 3
  # centered at (0, 0)

#midPointCircleDraw(13, 13, 3)



'\n        # Initializing the value of P\n        P = 1 - ratio\n        print(P)\n \n        while x > y:\n        \n            y += 1\n            \n            # Mid-point inside or on the perimeter\n            if P <= 0:\n                P = P + 2 * y + 1\n                \n                \n            # Mid-point outside the perimeter\n            else:        \n                x -= 1\n                P = P + 2 * y - 2 * x + 1\n            \n            # All the perimeter points have\n            # already been printed\n            if (x < y):\n                break\n            \n            # Printing the generated point its reflection\n            # in the other octants after translation\n        \n            ROUNDABOUT.append((x + x_centre, y + y_centre))\n            print("(", x + x_centre, ", ",\n                     y + y_centre, ")",\n                    sep = "", end = "")\n            \n            ROUNDABOUT.append((-x + x_centre, y + y_centre))\n            ROUND

In [196]:
class CleaningRoomsModel(ap.Model):
    def setup(self):
        
        # Create agents
    
        self.cars=ap.AgentList(self,2)
        #road=ap.AgentList(self,len(ROUNDABOUT))
        roadX=ap.AgentList(self,len(POSITIONS_X))
        roadY=ap.AgentList(self,len(POSITIONS_Y))
        

        # Create grid
        self.city = ap.Grid(self, [self.p.size] * 2, track_empty=True)
        
        self.city.add_agents(self.cars, [(26,13),(13,26)])
        #self.city.add_agents(road, positions=ROUNDABOUT ,empty=False)
        self.city.add_agents(roadX, positions=POSITIONS_X ,empty=False)
        self.city.add_agents(roadY, positions=POSITIONS_Y, empty=False)

        # Agent type attribute
        
        #0: car 0
        #1: untravelled road
        #2: travelled road
        #7: car 1

        self.cars.type_agent = 0
        #road.type_agent = 1
        roadX.type_agent = 1
        roadY.type_agent = 1

        #0: vertical
        #1: horizontal 
        self.cars[0].type_car = 0
        self.cars[1].type_car = 1
        
        self.cars[0].type_agent = 0
        self.cars[1].type_agent = 7

        # Road direction attribute
        #5: is horizontal
        #6: is vertical
        #road.road_direction = 5
        roadX.road_direction = 5
        roadY.road_direction = 6
        
        #road.road_type = 1
        roadX.road_type = 2
        roadY.road_type = 3

    def step(self):
        cars = self.cars


        for car in cars:
            old_position=self.city.positions[car]
            
            '''
            road_neighbors = []
            for neighbor in self.city.neighbors(car):
              if neighbor.type_agent == 1:
                road_neighbors.append(self.city.positions[neighbor])
            
            if road_neighbors.size() == 1:
              new_position = road_neighbors[0]

            elif road_neighbors.size() > 1:
              for neighbor in road_neighbors:
                if car.type_car == 0:
            '''     
                                                

            for neighbor in self.city.neighbors(car):
                new_position = self.city.positions[neighbor]
                if new_position[0] == 12 and new_position[1] == 14 or new_position[0] == 13 and new_position[1] == 12:
                    new_position = (13,13)

                if neighbor.type_agent == 1:   
                    self.city.move_to(car, new_position)
                    neighbor.type_agent = 2
                    break
                if neighbor.type_agent == 0 or neighbor.type_agent == 7:
                    if car.type_agent < neighbor.type_agent:
                        self.city.move_to(car, new_position)
                        neighbor.type_agent = 2
                        break
                    break 

      
'''

                if neighbor.type_agent == 1 and sigb.type_agent == 3 and car.type_car == 0:
                      self.city.move_to(car, new_position)
                      neighbor.type_agent = 2
                      break

                if neighbor.type_agent == 1 and sigc.type_agent == 3 and car.type_car == 1:
                    self.city.move_to(car, new_position)
                    neighbor.type_agent = 2                    
                    break
'''
                
                


'\n\n                if neighbor.type_agent == 1 and sigb.type_agent == 3 and car.type_car == 0:\n                      self.city.move_to(car, new_position)\n                      neighbor.type_agent = 2\n                      break\n\n                if neighbor.type_agent == 1 and sigc.type_agent == 3 and car.type_car == 1:\n                    self.city.move_to(car, new_position)\n                    neighbor.type_agent = 2                    \n                    break\n'

In [197]:
# Define parameters
parameters={
    'size':27,
    'steps':50,
}
# Create single-run animation with custom colors
def animation_plot (model, ax):
    attr_grid=model.city.attr_grid('type_agent')
    color_dict = {0:'#ff00ea', 1:'#808080', 2: '#ADFFBF',3:'#15b032',4:'#eb361a', 7:'#0062ff', 8: '#f5dd42', None: '#497a41'}
    ap.gridplot (attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Traffic simulation\n"
    f"Time-step: {model.t}")
fig, ax = plt. subplots()
model=CleaningRoomsModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))